In [22]:
import torch
import wandb
import pprint
import numpy as np

In [44]:
import os
from tqdm.notebook import tqdm
os.environ["http_proxy"]="http://proxy61.iitd.ac.in:3128"
os.environ["https_proxy"]="http://proxy61.iitd.ac.in:3128"

In [45]:
track_columns = ['new node test average_precision',
 'train roc_auc',
 'new node val first_3_roc_auc',
 'new node test first_3_roc_auc',
 'new node test first_3_average_precision',
 'new node val average_precision',
 'new node val first_1_average_precision',
 'new node test first_1_average_precision',
 'new node test first_1_roc_auc',
 'new node test first_10_roc_auc',
 'new node val first_3_average_precision',
 'val average_precision',
 'val roc_auc',
 'new node val roc_auc',
 'new node val first_10_average_precision',
 'new node test roc_auc',
 'train average_precision',
 'test roc_auc',
 'new node val first_1_roc_auc',
 'new node val first_10_roc_auc',
 'test average_precision',
 'new node test first_10_average_precision']

In [52]:
import pandas as pd 
import wandb
api = wandb.Api()

runs = api.runs("fb-graph-proj/fb-graph-proj-dyglib")

for run in tqdm(runs): 
    for key in track_columns:
        hist = run.history(samples = 200, keys = [key])[key]
        if type(hist) == float:
            print(f'From {run.summary[key]} to {np.nanmean(hist)}')
        run.summary[key] = np.nanmean(hist)
    run.summary.update()

  0%|          | 0/94 [00:00<?, ?it/s]

KeyError: 'new node val first_3_roc_auc'

In [ ]:
import pandas as pd 
import wandb
api = wandb.Api()

# Project is specified by <entity/project-name>
runs = api.runs("fb-graph-proj/fb-graph-proj-dyglib")

for run in tqdm(runs): 
    # .summary contains the output keys/values for metrics like accuracy.
    #  We call ._json_dict to omit large files 
    for key in run.summary.keys():
        hist = run.history(samples = 200, keys = [key])[key]
        if type(hist) == float:
            print(f'From {run.summary[key]} to {np.nanmean(hist)}')
        else:
            print(hist)
        run.summary[key] = np.nanmean(hist)
    run.summary.update()

In [ ]:
col = track_columns[0]
hist = runs[2].history(samples = 200)[col]
summary = runs[2].summary[col]
hist[199], np.nanmean(hist), summary

(0.6307716525833154, 0.74466270894532, 0.6307716525833154)

['new node test average_precision',
 'train roc_auc',
 'new node val first_3_roc_auc',
 'new node test first_3_roc_auc',
 'new node test first_3_average_precision',
 'new node val average_precision',
 'new node val first_1_average_precision',
 'new node test first_1_average_precision',
 'new node test first_1_roc_auc',
 'new node test first_10_roc_auc',
 'new node val first_3_average_precision',
 'val average_precision',
 'val roc_auc',
 'new node val roc_auc',
 'new node val first_10_average_precision',
 'new node test roc_auc',
 'train average_precision',
 'test roc_auc',
 'new node val first_1_roc_auc',
 'new node val first_10_roc_auc',
 'test average_precision',
 'new node test first_10_average_precision']

In [ ]:
def get_average_over_last_five_epochs(run_id):
  # Takes wandb run id and returns average of last 5 readings of the following metrics
  # new node test average_precision
  # test average_precision
  # test roc_auc
  # new node test roc_auc
  api = wandb.Api()
  run = api.run(f"/fb-graph-proj/fb-graph-proj-dyglib/runs/{run_id}")
  index = torch.argmax(torch.tensor(run.history()["val average_precision"].tolist()))
  METRICS_LIST = ["new node test average_precision", "test average_precision", "test roc_auc", "new node test roc_auc"]
  avg_metric = {}
  for metric in METRICS_LIST:
    avg_metric[metric] = round(float(100 * torch.nanmean(torch.tensor(run.history()[metric].tolist()))), 2)
  print(run.config["dataset"])
  pprint.pprint(avg_metric)
  # return avg_metric

In [ ]:
get_average_over_last_five_epochs("p0ft9874")

Contacts
{'new node test average_precision': 85.31,
 'new node test roc_auc': 90.1,
 'test average_precision': 80.5,
 'test roc_auc': 87.03}


In [2]:
import torch

def vectorized_update_mem_2d(
    mem: torch.Tensor, non_zero_counts: torch.Tensor, update: torch.Tensor
) -> torch.Tensor:
    """
    Update a 2D tensor 'mem' by appending values from 'update' tensor to each 'row' of 'mem'.
    This function operates in a queue-like fashion for each row.
    If a row has zeros (indicated by non_zero_counts), the first zero is replaced by the corresponding value from 'update'.
    If a row is full, it is shifted one place to the left, and the corresponding value from 'update' is added to the end.

    Parameters:
    mem (torch.Tensor): A 2D tensor of shape (n, seq_len) representing the memory to be updated.
    non_zero_counts (torch.Tensor): A 1D tensor of shape (n,) containing the count of non-zero 'elements' in each 'row' of 'mem'.
                                    Each element should be an integer.
    update (torch.Tensor): A 1D tensor of shape (n,) containing the values to be appended to each 'row' of 'mem'.

    Returns:
    torch.Tensor: The updated memory tensor of shape (n, seq_len).

    Note:
    - 'n' is the number of sequences, and 'seq_len' is the sequence length.
    - This function assumes that 'mem' has been appropriately padded with zeros where necessary.
    - The function operates in-place on 'mem', and the updated 'mem' tensor is also returned.
    # Example usage
    n = 3
    seq_len = 5
    mem = torch.rand(n, seq_len)  # Example 2D tensor
    print(mem)
    non_zero_counts = torch.tensor([2, 3, 5])  # Counts of non-zero elements in each row
    update = torch.rand(n)  # Update tensor
    print(update)
    updated_mem = vectorized_update_mem_2d(mem, non_zero_counts, update)
    print(updated_mem)
    """

    n, seq_len = mem.shape

    # Mask for rows that are full and need shifting
    full_rows_mask = non_zero_counts >= seq_len
    non_full_rows_mask = non_zero_counts < seq_len

    # Shift the full rows
    mem[full_rows_mask] = torch.roll(mem[full_rows_mask], -1, dims=1)

    # Determine insertion index for each row
    insertion_index = non_zero_counts.clone()
    insertion_index[full_rows_mask] = seq_len - 1  # If row is full, set index to last position

    # Create a mask for insertion points
    insertion_mask = torch.zeros(n, seq_len, dtype=torch.bool, device=mem.device)
    insertion_mask.scatter_(1, insertion_index.unsqueeze(1), True)

    # Insert new values
    mem = torch.where(insertion_mask, update.unsqueeze(1), mem)
    non_zero_counts[non_full_rows_mask] += 1

    return mem, non_zero_counts

In [4]:
n = 3
seq_len = 5
mem = torch.rand(n, seq_len)  # Example 2D tensor
print(mem)
non_zero_counts = torch.tensor([2, 3, 5])  # Counts of non-zero elements in each row
update = torch.rand(n)  # Update tensor
print(update)
updated_mem = vectorized_update_mem_2d(mem, non_zero_counts, update)
print(updated_mem)

tensor([[0.3519, 0.5518, 0.8356, 0.7327, 0.7677],
        [0.9330, 0.7643, 0.3385, 0.2019, 0.0988],
        [0.8304, 0.8748, 0.6190, 0.1182, 0.4605]])
tensor([0.6483, 0.1658, 0.1275])
(tensor([[0.3519, 0.5518, 0.6483, 0.7327, 0.7677],
        [0.9330, 0.7643, 0.3385, 0.1658, 0.0988],
        [0.8748, 0.6190, 0.1182, 0.4605, 0.1275]]), tensor([3, 4, 5]))
